## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beringovskiy,RU,63.0500,179.3167,43.70,66,100,15.12,overcast clouds
1,1,Albany,US,42.6001,-73.9662,65.95,94,100,2.17,moderate rain
2,2,Avarua,CK,-21.2078,-159.7750,73.45,94,100,14.97,light rain
3,3,Butaritari,KI,3.0707,172.7902,81.00,81,100,17.65,light rain
4,4,Wabag,PG,-5.4781,143.7021,71.24,35,100,4.85,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
my_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
my_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Albany,US,42.6001,-73.9662,65.95,94,100,2.17,moderate rain
2,2,Avarua,CK,-21.2078,-159.7750,73.45,94,100,14.97,light rain
3,3,Butaritari,KI,3.0707,172.7902,81.00,81,100,17.65,light rain
4,4,Wabag,PG,-5.4781,143.7021,71.24,35,100,4.85,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,76.75,66,0,20.65,clear sky
6,6,San Patricio,US,28.0170,-97.5169,82.35,86,100,6.15,overcast clouds
7,7,North Bend,US,43.4065,-124.2243,84.24,75,0,21.85,clear sky
8,8,Kewanee,US,41.2456,-89.9248,71.47,87,28,8.77,scattered clouds
9,9,Souillac,MU,-20.5167,57.5167,70.18,78,40,10.36,scattered clouds
10,10,Hamilton,US,39.1834,-84.5333,75.76,92,75,3.44,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
my_cities_df.count()

City_ID                473
City                   473
Country                471
Lat                    473
Lng                    473
Max Temp               473
Humidity               473
Cloudiness             473
Wind Speed             473
Current Description    473
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = my_cities_df.dropna(how='any')
clean_df.count()

City_ID                471
City                   471
Country                471
Lat                    471
Lng                    471
Max Temp               471
Humidity               471
Cloudiness             471
Wind Speed             471
Current Description    471
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,65.95,moderate rain,42.6001,-73.9662,
2,Avarua,CK,73.45,light rain,-21.2078,-159.7750,
3,Butaritari,KI,81.00,light rain,3.0707,172.7902,
4,Wabag,PG,71.24,overcast clouds,-5.4781,143.7021,
5,Atuona,PF,76.75,clear sky,-9.8000,-139.0333,
6,San Patricio,US,82.35,overcast clouds,28.0170,-97.5169,
7,North Bend,US,84.24,clear sky,43.4065,-124.2243,
8,Kewanee,US,71.47,scattered clouds,41.2456,-89.9248,
9,Souillac,MU,70.18,scattered clouds,-20.5167,57.5167,
10,Hamilton,US,75.76,broken clouds,39.1834,-84.5333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,73.45,light rain,-21.2078,-159.7750,Paradise Inn
3,Butaritari,KI,81.00,light rain,3.0707,172.7902,Isles Sunset Lodge
4,Wabag,PG,71.24,overcast clouds,-5.4781,143.7021,Shalom Inn
5,Atuona,PF,76.75,clear sky,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
6,San Patricio,US,82.35,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
...,...,...,...,...,...,...,...
711,Jiexiu,CN,71.47,clear sky,37.0244,111.9125,IU Hotel Jinzhong Yuhua Road
713,Piacabucu,BR,72.34,clear sky,-10.4056,-36.4344,Piaçabuçu News
715,Kiunga,PG,83.08,overcast clouds,-6.1219,141.2906,Airport Lodge Kiunga
718,Ambilobe,MG,65.66,clear sky,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))